In [1]:
import requests
import os
import re
import random
import urllib.request

url_dict = {
    'shakespeare.txt': 'https://caltech-cs155.s3.us-east-2.amazonaws.com/miniprojects/project3/data/shakespeare.txt',
    'spenser.txt': 'https://caltech-cs155.s3.us-east-2.amazonaws.com/miniprojects/project3/data/spenser.txt',
    'syllable_dict.txt' : 'https://caltech-cs155.s3.us-east-2.amazonaws.com/miniprojects/project3/data/Syllable_dictionary.txt',
    'about_syllable_dict.docx' : 'https://caltech-cs155.s3.us-east-2.amazonaws.com/miniprojects/project3/data/syllable_dict_explanation.docx'
}

def download_file(file_path):
    url = url_dict[file_path]
    print('Start downloading...')
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024 * 1024):
                f.write(chunk)
    print('Complete')

download_file('shakespeare.txt')
download_file('spenser.txt')
download_file('syllable_dict.txt')
download_file('about_syllable_dict.docx')

Start downloading...
Complete
Start downloading...
Complete
Start downloading...
Complete
Start downloading...
Complete


In [2]:
def parse_reverse_line_w_rhyme(text):
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}
    rhyme_pairs = []

    #99 and 126 do not strictly follow format
    for i in range(154):
        if i+1 == 99 or i+1 == 126: continue
        sonnet_start = i * 15
        if 99 < i+1 < 126: sonnet_start += 1
        elif i+1 > 126: sonnet_start -= 1
        for j in range(3):
            rhyme_pairs.append([lines[sonnet_start+1+j*4][-1], lines[sonnet_start+1+j*4+2][-1]])
            rhyme_pairs.append([lines[sonnet_start+1+j*4+1][-1], lines[sonnet_start+1+j*4+3][-1]])
            for k in range(4):
                obs_elem = []
                line = lines[sonnet_start+1+j*4+k]
                for word in line:
                    word = re.sub(r'[^\w\']', '', word).lower()
                    if word not in obs_map:
                        obs_map[word] = obs_counter
                        obs_counter += 1
                    obs_elem.insert(0, obs_map[word])
                obs.append(obs_elem)
        rhyme_pairs.append([lines[sonnet_start+13][-1], lines[sonnet_start+14][-1]])
        for k in range(2):
            obs_elem = []
            line = lines[sonnet_start+13+k]
            for word in line:
                word = re.sub(r'[^\w\']', '', word).lower()
                if word not in obs_map:
                    obs_map[word] = obs_counter
                    obs_counter += 1
                obs_elem.insert(0, obs_map[word])
            obs.append(obs_elem)
    rhyme_pairs = [[re.sub(r'[^\w\']', '', w).lower() for w in p] for p in rhyme_pairs]
    
    return obs, obs_map, rhyme_pairs

In [3]:
from Emily_project3_helper import *
shakespeare = open("shakespeare.txt", "r").read()

KeyboardInterrupt: 

In [ ]:
reversed_lines_obs, word_map, rhymes = parse_reverse_line_w_rhyme(shakespeare)
print(rhymes)

[['increase', 'decease'], ['die', 'memory'], ['eyes', 'lies'], ['fuel', 'cruel'], ['ornament', 'content'], ['spring', 'niggarding'], ['be', 'thee'], ['brow', 'now'], ['field', 'held'], ['lies', 'eyes'], ['days', 'praise'], ['use', "excuse'"], ['mine', 'thine'], ['old', 'cold'], ['viewest', 'renewest'], ['another', 'mother'], ['womb', 'tomb'], ['husbandry', 'posterity'], ['thee', 'see'], ['prime', 'time'], ['be', 'thee'], ['spend', 'lend'], ['legacy', 'free'], ['abuse', 'use'], ['give', 'live'], ['alone', 'gone'], ['deceive', 'leave'], ['thee', 'be'], ['frame', 'same'], ['dwell', 'excel'], ['on', 'gone'], ['there', 'where'], ['left', 'bereft'], ['glass', 'was'], ['meet', 'sweet'], ['deface', 'place'], ['distilled', 'selfkilled'], ['usury', 'thee'], ['loan', 'one'], ['art', 'depart'], ['thee', 'posterity'], ['fair', 'heir'], ['light', 'sight'], ['eye', 'majesty'], ['hill', 'still'], ['age', 'pilgrimage'], ['car', 'are'], ['day', 'way'], ['noon', 'son'], ['sadly', 'gladly'], ['joy', 'anno

### Aside - NLTK-Based Supervised Learning

In [ ]:
# Why not supervised learning?
import nltk
# nltk.download('averaged_perceptron_tagger')
idx_to_word = obs_map_reverser(word_map)
tagged_lines = []
for line_r in reversed_lines_obs:
    line_r.reverse()
    line_words = []
    for obs in line_r:
        line_words.append(idx_to_word.get(obs))
    tagged_lines.append(nltk.pos_tag(line_words))

reversed_states = []
for tagged_line in tagged_lines:
    tagged_line.reverse()
    reversed_states.append([pair[1] for pair in tagged_line])

state_to_idx = {}
state_idx = 0
for line in reversed_states:
    for state in line:
        if state_to_idx.get(state) is None:
            state_to_idx.update({state: state_idx})
            state_idx += 1

reversed_states_num = []
for line in reversed_states:
    reversed_states_num.append([state_to_idx.get(state) for state in line])

In [ ]:
from set6hmm import supervised_HMM
half_supervised_HMM = supervised_HMM(X=reversed_lines_obs, Y=reversed_states_num)
half_supervised_HMM.unsupervised_learning(reversed_lines_obs, 3)

hmm_rhyme_15_hidden = unsupervised_HMM(reversed_lines_obs, 15, 100)

epoch 1/3
epoch 2/3
epoch 3/3
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30


KeyboardInterrupt: 

In [ ]:
def get_state(O, obs, L):
    prob = [O[i][obs] for i in range(len(O))]
    prob = [p/sum(prob) for p in prob]
    return np.random.choice(L, p=prob)

from poem_utils import parse_syllable_dict, line_syllable_count

def get_line_w_ending(hmm, obs_map, end):
    obs = obs_map[end]
    obs_map_r = obs_map_reverser(obs_map)
    init_s = get_state(hmm.O, obs, hmm.L)
    states = [init_s]
    sentence = [end]
    for _ in range(7):
        s = np.random.choice(range(hmm.L), p=hmm.A[states[-1]])
        states.append(s)
        e = np.random.choice(range(hmm.D), p=hmm.O[s])
        sentence.append(obs_map_r[e])
    return sentence

syllable_dict = parse_syllable_dict()

def get_line_w_ending_syllables(hmm, obs_map, end, n_syllables=10):
    line = get_line_w_ending(hmm, obs_map, end)
    while n_syllables not in line_syllable_count(line, syllable_dict):
        line = get_line_w_ending(hmm, obs_map, end)
    return ' '.join(line).capitalize()

def rhyme_sample_sonnet(hmm, obs_map, rhymes, seed=None):
    rng = np.random.default_rng(seed=seed)
    sonnet = []
    for _ in range(3):
        r = rng.choice(len(rhymes), 2)
        end_words = [rhymes[r[0]][0], rhymes[r[1]][0], rhymes[r[0]][1], rhymes[r[1]][1]]
        stanza = [get_line_w_ending_syllables(hmm, obs_map, end) for end in end_words]
        stanza.append('\n')
        sonnet.append('\n'.join(stanza))
    rhyme_choice = rng.choice(len(rhymes))
    stanza = [get_line_w_ending_syllables(hmm, obs_map, rhymes[rhyme_choice][0]), 
              get_line_w_ending_syllables(hmm, obs_map, rhymes[rhyme_choice][1])]
    sonnet.append('\n'.join(stanza))
    
    return '\n'.join(sonnet)

In [ ]:
print(rhyme_sample_sonnet(hmm_rhyme_15_hidden, word_map, rhymes))

Part all he tell eyes find memory art
Of in this alone he although fair call
Thy string will gentle silence the whom heart
See assured that false of fears i fall


Under or chance my in decay my day
I there and come acceptance art of not
Bones thee in some beauty whose three away
Nothing fair fury but or gain bears forgot


Let 'gainst perceiv'st of farthest deep heaven old
Nothing kind you compiled not resort two press
My hath to yellow should from old behold
Hope none often lacked my best thy express


The great loving face alter then to foiled
Thy pity his and truly new me toiled


In [ ]:
wordclouds = states_to_wordclouds(hmm_rhyme_15_hidden, word_map)

In [ ]:
print(rhyme_sample_sonnet(half_supervised_HMM, word_map, rhymes))

Winter's mended and that should love faults gone
Dateless of in so tomorrow nay old o'er
Not not speak the have so public alone
Bear day i mistake was 'tis as before


Beauteous bears being lease they that takes lies
Did by plague lines chide eternal if here
Is sword night as am be and subtleties
The or harsh of to being but uprear


Sorrow up dulness wronk with side from thee
Are beauty judgment hast muse you face state
Such it prepare past now why lovely me
Cheek that not thy beauties nor belongs fate


And yet and their of batt'ring cannot thee
I where herd fountains gentle in thy me
